<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and Prepare Dataset

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
replace hw1-files/q3/patches.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hw1-files/q3/patches.csv  A

  inflating: hw1-files/q3/lsh.py     
  inflating: hw1-files/q1/dataset1.txt  
  inflating: hw1-files/q2/games_library.txt  


In [3]:
import pandas as pd
dataset = pd.read_csv("/content/hw1-files/q2/games_library.txt", header=None)

In [5]:
dataset[0] = dataset[0].apply(lambda x: x.split(" "))

In [6]:
dataset

,0
0,"[Far_Cry_6, God_of_War, Gran_Turismo_7, GTA_V,..."
1,"[UNCHARTED_4, God_of_War, FIFA_22, Dying_Light_2]"
2,"[Assassin's_Creed_Odyssey, God_of_War, UNCHART..."
3,"[DAYS_GONE, GTA_V, The_Last_of_Us_Part_II, God..."
4,"[Horizon_Forbidden_West, Gran_Turismo_7, GTA_V..."
...,...
39995,"[Dying_Light_2, The_Last_of_Us_Part_II, ARK:Su..."
39996,"[Assassin's_Creed_Odyssey, ARK:Survival_Evolve..."
39997,"[UNCHARTED_4, MAFIA_Trilogy, Dying_Light_2, Gh..."
39998,"[GTA_V, The_Last_of_Us_Part_II, It_Takes_Two, ..."


# Pass 1

In [6]:
SUPPORT_THRESHOLD = 100

In [7]:
support_unit = {}

for _, game_list in dataset.iterrows():
  for name in game_list[0]:
    if support_unit.get(name) is None:
      support_unit[name] = 0

    support_unit[name] += 1

In [8]:
support_unit

{'ARK:Survival_Evolved': 19587,
 'A_Way_Out': 19709,
 "Assassin's_Creed_Odyssey": 19700,
 'Control': 19653,
 'DAYS_GONE': 19838,
 'Death_Stranding': 19689,
 'Dying_Light_2': 19652,
 'FIFA_22': 32457,
 'Far_Cry_6': 19549,
 'GTA_V': 32581,
 'Ghost_of_Tsushima': 19639,
 'God_of_War': 32390,
 'Gran_Turismo_7': 19667,
 'Horizon_Forbidden_West': 19788,
 'It_Takes_Two': 19808,
 'MAFIA_Trilogy': 19782,
 'Red_Dead_Redemption_2': 19756,
 'Spider-Man:Miles_Morales': 19819,
 'The_Last_of_Us_Part_II': 32583,
 'UNCHARTED_4': 32583}

In [11]:
frequent_unit = []

for game_name in support_unit:
  if support_unit[game_name] >= SUPPORT_THRESHOLD:
    frequent_unit.append(game_name)

frequent_unit = sorted(frequent_unit)

In [12]:
frequent_unit

['ARK:Survival_Evolved',
 'A_Way_Out',
 "Assassin's_Creed_Odyssey",
 'Control',
 'DAYS_GONE',
 'Death_Stranding',
 'Dying_Light_2',
 'FIFA_22',
 'Far_Cry_6',
 'GTA_V',
 'Ghost_of_Tsushima',
 'God_of_War',
 'Gran_Turismo_7',
 'Horizon_Forbidden_West',
 'It_Takes_Two',
 'MAFIA_Trilogy',
 'Red_Dead_Redemption_2',
 'Spider-Man:Miles_Morales',
 'The_Last_of_Us_Part_II',
 'UNCHARTED_4']

# Pass 2

In [13]:
support_pair = {}
for i, game_name1 in enumerate(frequent_unit):
  for j, game_name2 in enumerate(frequent_unit):
    if i < j:
        support_pair[(game_name1, game_name2)] = 0

In [14]:
for _, game_list in dataset.iterrows():
  game_list_names = sorted(game_list[0])

  for i, game_name1 in enumerate(game_list_names):
    for j, game_name2 in enumerate(game_list_names):
      if i<j and support_pair.get((game_name1, game_name2)) is not None:
        support_pair[(game_name1, game_name2)] += 1

In [18]:
frequent_pair = []

for pair in support_pair:
  if support_pair[pair] >= SUPPORT_THRESHOLD:
    frequent_pair.append(pair)

frequent_pair = sorted(frequent_pair)

In [19]:
def calculate_confidence_pair(left, right):
  if right < left:
    return support_pair[(right, left)] / support_unit[left]
  else:
    return support_pair[(left, right)] / support_unit[left]

In [20]:
confidence_pair = []

for pair in frequent_pair:
  confidence_pair.append({"rule_left": pair[0],"rule_right": pair[1], "confidence": calculate_confidence_pair(left=pair[0], right=pair[1])})
  confidence_pair.append({"rule_left": pair[1],"rule_right": pair[0], "confidence": calculate_confidence_pair(left=pair[1], right=pair[0])})

In [21]:
sorted(confidence_pair, key=lambda item: (-item["confidence"], (item["rule_left"], item["rule_right"])))[:5]

[{'confidence': 0.9150213718705476,
  'rule_left': 'Dying_Light_2',
  'rule_right': 'The_Last_of_Us_Part_II'},
 {'confidence': 0.9136672282636442,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'The_Last_of_Us_Part_II'},
 {'confidence': 0.9136161739929545,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'GTA_V'},
 {'confidence': 0.9125440343084699,
  'rule_left': 'ARK:Survival_Evolved',
  'rule_right': 'UNCHARTED_4'},
 {'confidence': 0.9119608941392128,
  'rule_left': 'Ghost_of_Tsushima',
  'rule_right': 'UNCHARTED_4'}]

# Pass 3